In [1]:
%pip install cmem-cmempy llama-index python-dotenv llama-index-vector-stores-postgres

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext dotenv
%dotenv

In [3]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
#!docker pull ankane/pgvector
#!docker run -e POSTGRES_USER=eccenca -e POSTGRES_PASSWORD=eccenca -e POSTGRES_DB=eccenca --name pgvector-eccenca -p 5432:5432 ankane/pgvector

In [5]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database="eccenca",
    host="localhost",
    password="eccenca",
    port="5432",
    user="eccenca",
    table_name="prod-inst",
    embed_dim=1536,  # openai embedding dimension
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine(similarity_top_k=30)

answer = query_engine.query("What can you tell me about Dietlinde Boehme? Give me as many information and details as possible. Format the result nicely as Markdown. If sensible use list-style format. Also add a textual summary.")
print("Answer:\n")
print(answer)
print("\nSource nodes:\n")
for node in answer.source_nodes:
    print(node)

PG Setup: Error creating HNSW index: (psycopg2.errors.SyntaxError) syntax error at or near "-"
LINE 1: CREATE INDEX IF NOT EXISTS data_prod-inst_embedding_idx ON p...
                                            ^

[SQL: CREATE INDEX IF NOT EXISTS data_prod-inst_embedding_idx ON public.data_prod-inst USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64)]
(Background on this error at: https://sqlalche.me/e/20/f405)


Answer:

- **Name:** Dietlinde Boehme
- **Email:** Dietlinde.Boehme@company.org
- **Member of:** Marketing
- **Address:** Biebricher Allee 880, 65826 Neukalen
- **Phone number:** (06582) 6550636
- **Manages:** Liese Adam, Valda Everhart, Lambert Faust, Yanka Schreiber, Baldwin Dirksen, Bert Blumstein, Henny Foth, Kevin Feigenbaum, Sabrina Geiger
- **Product Manager for:** Planck Dipole Driver Crystal, Transformer Inductor Encoder, Crystal Strain, Network Multiplexer, Field-effect Network Rheostat
- **Area of Expertise:** Breaker

### Summary:
Dietlinde Boehme is a member of the Marketing department. She manages a team of individuals and is the product manager for various products including Planck Dipole Driver Crystal, Transformer Inductor Encoder, Crystal Strain, Network Multiplexer, and Field-effect Network Rheostat. Her area of expertise lies in Breaker.

Source nodes:

Node ID: 46ab8c15-18e7-4177-8c89-11e7c588d5ad
Text: Dietlinde Boehme name Dietlinde Boehme
Score:  0.868

Node ID: